In [1]:
!pip install flask

In [11]:
import ccxt
print('CCXT Version:', ccxt.__version__)
exchange = ccxt.binance({
    'enableRateLimit': True,
    'apiKey': '6f451265b0590b2776adc8079c3875460c0b11177dd690f66a780f37c39a245f',
    'secret': '476acb93f59148c133c5ed93b33e456f0e38eb1e530b505d139e34c4eb9b3ea3',
    'options': {
        'defaultType': 'future',
    },
    'hedgeMode': True
})

exchange.set_sandbox_mode(True)

markets = exchange.load_markets()

exchange.verbose = True

balance = exchange.fetch_balance()

CCXT Version: 1.65.15

fetch Request: binance GET https://testnet.binancefuture.com/fapi/v2/account?timestamp=1641718507251&recvWindow=5000&signature=179bfbde7155947b7683f31c32d2d6de120ac500c2b53b1ad728ff3b64c70004 RequestHeaders: {'X-MBX-APIKEY': '6f451265b0590b2776adc8079c3875460c0b11177dd690f66a780f37c39a245f', 'User-Agent': 'python-requests/2.26.0', 'Accept-Encoding': 'gzip, deflate'} RequestBody: None

fetch Response: binance GET https://testnet.binancefuture.com/fapi/v2/account?timestamp=1641718507251&recvWindow=5000&signature=179bfbde7155947b7683f31c32d2d6de120ac500c2b53b1ad728ff3b64c70004 200 ResponseHeaders: {'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Sun, 09 Jan 2022 08:55:07 GMT', 'Server': 'Bengine', 'Vary': 'Accept-Encoding', 'X-MBX-USED-WEIGHT-1M': '6', 'x-response-time': '3ms', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Access-Control-Allow-Credentia

In [ ]:
response = exchange.fapiPrivate_post_positionside_dual({'dualSidePosition': True, })

In [56]:
market = exchange.market('LUNA/USDT')
response = exchange.fapiPrivate_post_margintype({'symbol': market['id'],'marginType': 'ISOLATED',})


fetch Request: binance POST https://testnet.binancefuture.com/fapi/v1/marginType RequestHeaders: {'X-MBX-APIKEY': '6f451265b0590b2776adc8079c3875460c0b11177dd690f66a780f37c39a245f', 'Content-Type': 'application/x-www-form-urlencoded', 'User-Agent': 'python-requests/2.26.0', 'Accept-Encoding': 'gzip, deflate'} RequestBody: timestamp=1641728121737&recvWindow=5000&symbol=LUNAUSDT&marginType=ISOLATED&signature=c29e69989506b13b22fdd180676d59d6a9f580c9f2d60ff14de4992c9142469d

fetch Response: binance POST https://testnet.binancefuture.com/fapi/v1/marginType 200 ResponseHeaders: {'Content-Type': 'application/json', 'Content-Length': '30', 'Connection': 'keep-alive', 'Date': 'Sun, 09 Jan 2022 11:35:22 GMT', 'Server': 'Bengine', 'X-MBX-USED-WEIGHT-1M': '3', 'x-response-time': '2ms', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': '*', 'Content-Security-Policy': "de

In [60]:
market = exchange.market('LUNA/USDT')
exchange.fapiPrivate_post_leverage({  # https://github.com/ccxt/ccxt/wiki/Manual#implicit-api-methods
    'symbol': market['id'],  # https://github.com/ccxt/ccxt/wiki/Manual#symbols-and-market-ids
    'leverage': 20,  # target initial leverage, int from 1 to 125
})


fetch Request: binance POST https://testnet.binancefuture.com/fapi/v1/leverage RequestHeaders: {'X-MBX-APIKEY': '6f451265b0590b2776adc8079c3875460c0b11177dd690f66a780f37c39a245f', 'Content-Type': 'application/x-www-form-urlencoded', 'User-Agent': 'python-requests/2.26.0', 'Accept-Encoding': 'gzip, deflate'} RequestBody: timestamp=1641728177562&recvWindow=5000&symbol=LUNAUSDT&leverage=20&signature=974378ad645b317a450b6dc26ab0d2b6e073d755391c2baea8e9f6a58d37037b

fetch Response: binance POST https://testnet.binancefuture.com/fapi/v1/leverage 200 ResponseHeaders: {'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Sun, 09 Jan 2022 11:36:18 GMT', 'Server': 'Bengine', 'Vary': 'Accept-Encoding', 'X-MBX-USED-WEIGHT-1M': '2', 'x-response-time': '5ms', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': '*', 'Conten

{'symbol': 'LUNAUSDT', 'leverage': '20', 'maxNotionalValue': '25000'}

In [61]:
def is_buy_open(exchange,symbol):
    response = exchange.fetch_balance()
    k = [t for t in response['info']['positions'] if t['symbol'] == symbol]
    for a in k:
        if a['positionSide'] == 'LONG' and float(a['positionAmt']) != 0:
            return True, float(a['positionAmt'])
    return False, 0

def is_sell_open(exchange,symbol):
    response = exchange.fetch_balance()
    k = [t for t in response['info']['positions'] if t['symbol'] == symbol]
    for a in k:
        if a['positionSide'] == 'SHORT' and float(a['positionAmt']) != 0:
            return True, float(a['positionAmt'])
    return False, 0

def get_max_qty(symbol,qty):
    if (symbol == 'BTCUSDT'):
        max_qty = 120
        return max_qty
    if (symbol == 'ETHUSDT'):
        max_qty = 2000
        return max_qty
    if (symbol == 'SOLUSDT'):
        max_qty = 9000
        return max_qty
    if (symbol == 'LUNAUSDT'):
        max_qty = 15000
        return max_qty
    if (symbol == 'DOTUSDT'):
        max_qty = 50000
        return max_qty
    return qty

def limit_buy(exchange, symbol, order_type, at_price, side='buy'):
    equity = float(exchange.fetch_balance().get('USDT').get('free'))
    qty = ((equity/10.0) * 20) / at_price
    max_qty = get_max_qty(symbol,qty)
    if qty > max_qty:
        qty = max_qty
    order_params = {'positionSide':'LONG', 'recvWindow': 10000000}
    main_order = exchange.create_order(symbol, 'limit', 'buy', qty, at_price, order_params)
    return main_order

def limit_sell(exchange, symbol, order_type, at_price, side='sell'):
    equity = float(exchange.fetch_balance().get('USDT').get('free'))
    qty = ((equity/10.0) * 20) / at_price
    max_qty = get_max_qty(symbol,qty)
    if qty > max_qty:
        qty = max_qty
    order_params = {'positionSide':'SHORT', 'recvWindow': 10000000}
    main_order = exchange.create_order(symbol, 'limit', 'sell', qty, at_price, order_params)
    return main_order


def close_short_unfilled_orders(exchange, symbol):
    res = exchange.fetch_open_orders(symbol)
    if res is not None:
        for r in res:
            if r.get('info').get('positionSide') == 'SHORT' and r.get('filled') == 0:
                exchange.cancel_order(r.get('id'),r.get('symbol'), params={'recvWindow': 10000000})

def close_long_unfilled_orders(exchange, symbol):
    res = exchange.fetch_open_orders(symbol)
    if res is not None:
        for r in res:
            if r.get('info').get('positionSide') == 'LONG' and r.get('filled') == 0:
                exchange.cancel_order(r.get('id'),r.get('symbol'), params={'recvWindow': 10000000})

def close_long(exchange, symbol):
    is_buy, amount = is_buy_open(exchange, symbol)
    if is_buy:
        exchange.create_order(symbol, 'market', "sell", abs(amount), None, params={'positionSide':'LONG', 'recvWindow': 10000000})

def close_short(exchange, symbol):
    is_sell, amount = is_sell_open(exchange, symbol)
    if is_sell:
        exchange.create_order(symbol, 'market', "buy", abs(amount), None, params={'positionSide':'SHORT', 'recvWindow': 10000000})
        


In [66]:
from flask import Flask
from flask import request
from IPython.display import display, clear_output
import json

app = Flask(__name__)

buy_main_order = None
sell_main_order = None

@app.route('/webhook', methods=['POST'])
def tradingview_webhook():
    clear_output(wait=True)
    data = json.loads(request.data)
    symbol = 'BTCUSDT'
    if data['Action'] == 'buy':
        if data['Position_Size'] == 0:
            close_short(exchange, symbol)
            close_short_unfilled_orders(exchange, symbol)
        if data['Position_Size'] != 0:
            buy_main_order = limit_buy(exchange, symbol, 'limit', data['Price'], 'buy')
            close_short_unfilled_orders(exchange, symbol)
    if data['Action'] == 'sell':
        if data['Position_Size'] == 0:
            close_long(exchange, symbol)
            close_long_unfilled_orders(exchange, symbol)
        if data['Position_Size'] != 0:
            sell_main_order = limit_sell(exchange, symbol, 'limit', data['Price'], 'sell')
            close_long_unfilled_orders(exchange, symbol)
    print(data)
    return {"Status":"Success"}
  
# main driver function
if __name__ == '__main__':
    app.run(port=80)


fetch Request: binance GET https://testnet.binancefuture.com/fapi/v2/account?timestamp=1641817961364&recvWindow=5000&signature=e3851f14d8f1fdf426abab0bf5cfd526089cd181eedf67d7d811a9bcc072776c RequestHeaders: {'X-MBX-APIKEY': '6f451265b0590b2776adc8079c3875460c0b11177dd690f66a780f37c39a245f', 'User-Agent': 'python-requests/2.26.0', 'Accept-Encoding': 'gzip, deflate'} RequestBody: None

fetch Response: binance GET https://testnet.binancefuture.com/fapi/v2/account?timestamp=1641817961364&recvWindow=5000&signature=e3851f14d8f1fdf426abab0bf5cfd526089cd181eedf67d7d811a9bcc072776c 200 ResponseHeaders: {'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Mon, 10 Jan 2022 12:32:42 GMT', 'Server': 'Bengine', 'Vary': 'Accept-Encoding', 'X-MBX-USED-WEIGHT-1M': '5', 'x-response-time': '3ms', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Access-Control-Allow-Credentials': 'true', 'Access-C


fetch Response: binance GET https://testnet.binancefuture.com/fapi/v2/account?timestamp=1641817961306&recvWindow=5000&signature=79c58f6786a4611d243c0d368269a34aad19059e710c994a5be2679328fbbb29 200 ResponseHeaders: {'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Mon, 10 Jan 2022 12:32:42 GMT', 'Server': 'Bengine', 'Vary': 'Accept-Encoding', 'X-MBX-USED-WEIGHT-1M': '10', 'x-response-time': '2ms', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': '*', 'Content-Security-Policy': "default-src 'self'", 'cache-control': 'no-cache, no-store, must-revalidate', 'expires': '0', 'pragma': 'no-cache', 'strict-transport-security': 'max-age=31536000; includeSubdomains', 'x-content-type-options': 'nosniff', 'x-frame-options': 'SAMEORIGIN', 'x-xss-protection': '1; mode=block', 'Content-Encoding': 'gzip', 'X-Cache': 'M


fetch Response: binance POST https://testnet.binancefuture.com/fapi/v1/order 200 ResponseHeaders: {'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Mon, 10 Jan 2022 12:32:43 GMT', 'Server': 'Bengine', 'Vary': 'Accept-Encoding', 'X-MBX-USED-WEIGHT-1M': '10', 'X-MBX-ORDER-COUNT-10S': '0', 'X-MBX-ORDER-COUNT-1M': '1', 'x-response-time': '3ms', 'X-Frame-Options': 'SAMEORIGIN', 'X-Xss-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'Referrer-Policy': 'origin-when-cross-origin', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains', 'Content-Encoding': 'gzip', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 42a90eee92b9e5976aea125e8f9d2f9a.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'LHR50-C1', 'X-Amz-Cf-Id': 'iuXmoY2NIBxtuNM1fseZb8wTDtOQtDPUyqbPnvTlJbmO3s2zSQ5TfQ=='} ResponseBody: {"orderId":2964357575,"symbol":"BTCUSDT","status":"NEW","clientOrderId":"x-xcKtGhcuaaa13e5a036d1c618dac67","price":"41400",

127.0.0.1 - - [10/Jan/2022 18:02:43] "POST /webhook HTTP/1.1" 200 -



fetch Response: binance GET https://testnet.binancefuture.com/fapi/v1/openOrders?timestamp=1641817962948&recvWindow=5000&symbol=BTCUSDT&signature=9db44332a879b46cace58562955921e37f384d3fced7cafd0d3af2d3a0e058bc 200 ResponseHeaders: {'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Mon, 10 Jan 2022 12:32:44 GMT', 'Server': 'Bengine', 'Vary': 'Accept-Encoding', 'X-MBX-USED-WEIGHT-1M': '11', 'x-response-time': '0ms', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': '*', 'Content-Security-Policy': "default-src 'self'", 'cache-control': 'no-cache, no-store, must-revalidate', 'expires': '0', 'pragma': 'no-cache', 'strict-transport-security': 'max-age=31536000; includeSubdomains', 'x-content-type-options': 'nosniff', 'x-frame-options': 'SAMEORIGIN', 'x-xss-protection': '1; mode=block', 'Content-Encoding': 'gz

127.0.0.1 - - [10/Jan/2022 18:02:43] "POST /webhook HTTP/1.1" 200 -



fetch Response: binance GET https://testnet.binancefuture.com/fapi/v1/openOrders?timestamp=1641817963169&recvWindow=5000&symbol=BTCUSDT&signature=e9433ab09e342bf111eb4efd36eb5e86903dd982efe6b2af97c3e6bbfc8383d9 200 ResponseHeaders: {'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Mon, 10 Jan 2022 12:32:44 GMT', 'Server': 'Bengine', 'Vary': 'Accept-Encoding', 'X-MBX-USED-WEIGHT-1M': '12', 'x-response-time': '0ms', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': '*', 'Content-Security-Policy': "default-src 'self'", 'cache-control': 'no-cache, no-store, must-revalidate', 'expires': '0', 'pragma': 'no-cache', 'strict-transport-security': 'max-age=31536000; includeSubdomains', 'x-content-type-options': 'nosniff', 'x-frame-options': 'SAMEORIGIN', 'x-xss-protection': '1; mode=block', 'Content-Encoding': 'gz